In [1]:
import os
os.chdir('../../')

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scr.util import *

In [5]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

In [6]:
df_train.isna().sum()

id                          0
Age                       100
TypeofContact               6
CityTier                    0
DurationOfPitch           121
Occupation                  0
Gender                      0
NumberOfPersonVisiting      0
NumberOfFollowups          33
ProductPitched              0
PreferredPropertyStar       0
NumberOfTrips              22
Passport                    0
PitchSatisfactionScore      0
Designation                 0
MonthlyIncome              56
customer_info               0
ProdTaken                   0
dtype: int64

# 今回調査する欠損値：Age, DurationOfPitch, NumberOfTrips, MonthlyIncome

In [7]:
df_train['Age'] = df_train['Age'].astype(str).apply(japanese_to_int)
df_test['Age'] = df_test['Age'].astype(str).apply(japanese_to_int)

df_train['TypeofContact'] = df_train['TypeofContact'].fillna('No')
df_test['TypeofContact'] = df_test['TypeofContact'].fillna('No')

df_train['DurationOfPitch'] = df_train['DurationOfPitch'].dropna().apply(convert_to_second)
df_test['DurationOfPitch'] = df_test['DurationOfPitch'].dropna().apply(convert_to_second)

df_train['Gender'] = df_train['Gender'].apply(normalize_gender)
df_test['Gender'] = df_test['Gender'].apply(normalize_gender)

df_train['NumberOfFollowups'] = df_train['NumberOfFollowups'].apply(lambda x: x if x < 100 else x / 100)
df_test['NumberOfFollowups'] = df_test['NumberOfFollowups'].apply(lambda x: x if x < 100 else x / 100)
df_train['NumberOfFollowups'] = df_train['NumberOfFollowups'].fillna(0)
df_test['NumberOfFollowups'] = df_test['NumberOfFollowups'].fillna(0)

df_train['ProductPitched'] = df_train['ProductPitched'].apply(normalize_product_pitched_1)
df_train['ProductPitched'] = df_train['ProductPitched'].apply(normalize_product_pitched_2)
df_test['ProductPitched'] = df_test['ProductPitched'].apply(normalize_product_pitched_1)
df_test['ProductPitched'] = df_test['ProductPitched'].apply(normalize_product_pitched_2)

df_train['NumberOfTrips'] = df_train['NumberOfTrips'].apply(normalize_trips)
df_test['NumberOfTrips'] = df_test['NumberOfTrips'].apply(normalize_trips)

df_train['Designation'] = df_train['Designation'].apply(normalize_designation_1)
df_train['Designation'] = df_train['Designation'].apply(normalize_designation_2)
df_test['Designation'] = df_test['Designation'].apply(normalize_designation_1)
df_test['Designation'] = df_test['Designation'].apply(normalize_designation_2)

df_train['MonthlyIncome'] = df_train['MonthlyIncome'].apply(normalize_monthly_income)
df_test['MonthlyIncome'] = df_test['MonthlyIncome'].apply(normalize_monthly_income)

df_train[['Marry', 'Car', 'Child']] = df_train['customer_info'].apply(divide_customer_info)
df_test[['Marry', 'Car', 'Child']] = df_test['customer_info'].apply(divide_customer_info)
df_train['Marry'] = df_train['Marry'].apply(normalize_info_1)
df_test['Marry'] = df_test['Marry'].apply(normalize_info_1)
df_train['Car'] = df_train['Car'].apply(normalize_info_2)
df_test['Car'] = df_test['Car'].apply(normalize_info_2)
df_train['Child'] = df_train['Child'].apply(normalize_info_3)
df_test['Child'] = df_test['Child'].apply(normalize_info_3)

In [8]:
# Age : 0
# NumberOfTrips : 'nan'

df_train.isna().sum()

id                          0
Age                         0
TypeofContact               0
CityTier                    0
DurationOfPitch           121
Occupation                  0
Gender                      0
NumberOfPersonVisiting      0
NumberOfFollowups           0
ProductPitched              0
PreferredPropertyStar       0
NumberOfTrips               0
Passport                    0
PitchSatisfactionScore      0
Designation                 0
MonthlyIncome              56
customer_info               0
ProdTaken                   0
Marry                       0
Car                         0
Child                       0
dtype: int64

In [9]:
# Age : 0
# NumberOfTrips : 'nan'

df_test.isna().sum()

id                          0
Age                         0
TypeofContact               0
CityTier                    0
DurationOfPitch           131
Occupation                  0
Gender                      0
NumberOfPersonVisiting      0
NumberOfFollowups           0
ProductPitched              0
PreferredPropertyStar       0
NumberOfTrips               0
Passport                    0
PitchSatisfactionScore      0
Designation                 0
MonthlyIncome              49
customer_info               0
Marry                       0
Car                         0
Child                       0
dtype: int64

In [11]:
drop_list = ['id', 'customer_info']

df_train = df_train.drop(columns=drop_list)
df_test = df_test.drop(columns=drop_list)

In [12]:
df_train.to_csv('data/null_survey/train_null.csv', index=False)
df_test.to_csv('data/null_survey/test_null.csv', index=False)

# Age

In [42]:
df_train = pd.read_csv('data/null_survey/train_null.csv')
df_test = pd.read_csv('data/null_survey/test_null.csv')

In [43]:
int_columns = ['TypeofContact','CityTier','Occupation','Gender','NumberOfPersonVisiting','NumberOfFollowups','ProductPitched','PreferredPropertyStar','Passport','Car','Child']

In [44]:
def age_to_agegroup(age):
    if age == 0:
        return np.nan
    elif age < 20:
        return "10s"
    elif age < 30:
        return "20s"
    elif age < 40:
        return "30s"
    elif age < 50:
        return "40s"
    elif age < 60:
        return "50s"
    else:
        return "60s"

df_train['AgeGroup'] = df_train['Age'].apply(age_to_agegroup)
df_test['AgeGroup'] = df_test['Age'].apply(age_to_agegroup)

In [67]:
for col in int_columns:
    grouped = df_train.groupby(by=['AgeGroup', col])['Age'].count()
    total_counts_by_agegroup = grouped.groupby(level=0).sum()
    percentage = grouped / total_counts_by_agegroup
    grouped = pd.concat([grouped, percentage], axis=1)
    grouped.columns = ['count', 'percent']
    print(grouped)

                          count   percent
AgeGroup TypeofContact                   
10s      Company Invited      5  0.277778
         Self Enquiry        13  0.722222
20s      Company Invited    213  0.386570
         Self Enquiry       338  0.613430
30s      Company Invited    485  0.391445
         No                   4  0.003228
         Self Enquiry       750  0.605327
40s      Company Invited    311  0.341383
         No                   2  0.002195
         Self Enquiry       598  0.656422
50s      Company Invited    226  0.350388
         Self Enquiry       419  0.649612
60s      Company Invited      8  0.320000
         Self Enquiry        17  0.680000
                   count   percent
AgeGroup CityTier                 
10s      1             8  0.444444
         2             9  0.500000
         3             1  0.055556
20s      1           176  0.319419
         2           298  0.540835
         3            77  0.139746
30s      1           512  0.413236
         2   

Index(['Age', 'TypeofContact', 'CityTier', 'DurationOfPitch', 'Occupation',
       'Gender', 'NumberOfPersonVisiting', 'NumberOfFollowups',
       'ProductPitched', 'PreferredPropertyStar', 'NumberOfTrips', 'Passport',
       'PitchSatisfactionScore', 'Designation', 'MonthlyIncome', 'ProdTaken',
       'Marry', 'Car', 'Child', 'AgeGroup'],
      dtype='object')

3488    61
3487    61
3486    61
3485    61
3484    61
3474    60
3464    60
3465    60
3466    60
3467    60
Name: Age, dtype: int64